In [ ]:
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D,MaxPool2D ,UpSampling2D, Flatten, Input
from keras.optimizers import SGD, Adam, Adadelta, Adagrad
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
import cv2
from math import log10,sqrt

In [ ]:
train_data=sorted(os.listdir('/kaggle/input/chest-xrays-radiopaedia/images/images'))
noise_data=sorted(os.listdir('/kaggle/input/chest-noisy/chest noisy-20230306T183108Z-001/chest noisy'))

In [ ]:
train_data_dir='/kaggle/input/chest-xrays-radiopaedia/images/images'
noise_data_dir='/kaggle/input/chest-noisy/chest noisy-20230306T183108Z-001/chest noisy'

In [ ]:
from tqdm.notebook import tqdm
train_image = []
for file in tqdm(sorted(os.listdir(train_data_dir))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(train_data_dir + '/' + file, target_size=(256,256),color_mode='grayscale')
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    train_image.append(image)

train_image = np.array(train_image)

In [ ]:
def plot_img(dataset):
  f,ax=plt.subplots(1,5)
  f.set_size_inches(40,20)
  for i in range(5,10):
    ax[i-5].imshow(dataset[i], cmap='gray')
  plt.show()

In [ ]:
noised_image = []
for file in tqdm(sorted(os.listdir(noise_data_dir))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(noise_data_dir + '/' + file, target_size=(256,256),color_mode='grayscale')
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    noised_image.append(image)

noised_image = np.array(noised_image)

In [ ]:
print(train_image.shape)

In [ ]:
plot_img(train_image)
# import matplotlib.pyplot as plt
#plt.imshow(train_image)
#plt.show()


In [ ]:
plot_img(noised_image)

In [ ]:
xnoised=noised_image[:400]
xtest=noised_image[400:]

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model

def autoencoder():
    # Input layer
    inputs = Input(shape=(256, 256, 1))

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)

    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    merge6 = concatenate([up6, conv4], axis=3)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    merge7 = concatenate([up7, conv3], axis=3)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    merge8 = concatenate([up8, conv2], axis=3)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)
    
    up9 = Conv2DTranspose(64, (2, 2), strides=(2,2), padding='same')(conv8)
    merge9 = concatenate([up9, conv1], axis=3)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)
    
    # Output layer
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

# Compile the model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [ ]:
model= autoencoder()
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
model = autoencoder()
model.compile(optimizer='adam', loss='mse')
plot_model(model, show_shapes=True)

In [ ]:
with tf.device('/device:GPU:0'):
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
    model.fit(xnoised, xnoised, epochs= 2, batch_size=1, validation_data=(xtest, xtest), callbacks=[early_stopping])

In [ ]:
xtrain= train_image[100:]

In [ ]:
pred= model.predict(xtest[:10])
def plot_predictions(y_true, y_pred):    
    f, ax = plt.subplots(4, 5)
    f.set_size_inches(10.5,7.5)
    for i in range(10):
        ax[0][i].imshow(np.reshape(xtrain[i], (256,256,1)), aspect='auto', cmap='gray')
        ax[1][i].imshow(np.reshape(y_true[i], (256,256,1)), aspect='auto', cmap='gray')
        ax[2][i].imshow(np.reshape(y_pred[i], (256,256,1)), aspect='auto', cmap='gray')
        ax[3][i].imshow(cv2.medianBlur(xtrain[i], (5)), aspect='auto', cmap='gray')
    plt.tight_layout()
plot_predictions(xtest[:10], pred[:10])

In [ ]:
median_blur = cv2.medianBlur(xtrain[0], (5))
gaussian_blur=cv2.GaussianBlur(xtrain[0],(5,5),0)
average_blur=cv2.blur(xtrain[0],(5,5))
bilateral_filter=cv2.bilateralFilter(xtrain[0],9,75,75)
f,ax=plt.subplots(1,5)
f.set_size_inches(40,20)
ax[0].imshow(pred[0].reshape(256,256,1), cmap='gray')
ax[0].set_title('Autoencoder Image')
ax[1].imshow(median_blur,cmap='gray')
ax[1].set_title('Median Filter')
ax[2].imshow(gaussian_blur,cmap='gray')
ax[2].set_title('Gaussian Filter')
ax[3].imshow(average_blur,cmap='gray')
ax[3].set_title('Average Filter')
ax[4].imshow(bilateral_filter,cmap='gray')
ax[4].set_title('Bilateral Filter')

In [ ]:
def PSNR(original, denoised): 
    mse = np.mean((original - denoised) ** 2) 
    if(mse == 0): 
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse)) 
    return psnr 

  
#value1 = PSNR(xtest[0], median_blur)
value2 = PSNR(xtrain[0], pred[0])
#value3 = PSNR(xtest[0], gaussian_blur)
#value4 = PSNR(xtest[0], average_blur)
#value5 = PSNR(xtest[0], bilateral_filter)
#value6 = PSNR(xtrain[0],pred[0])

print("PSNR values")
print(f"Autoencoder Image : {value2} dB")
#print(f"Median Filter Image : {value1} dB")
#print(f"Gaussian Filter Image : {value3} dB")
#print(f"Average Filter Image : {value4} dB")
#print(f"Bilateral Filter Image : {value5} dB")
#print(f"Noisy Image : {value6} dB")

In [ ]:
#pip install --upgrade scikit-imag

In [ ]:
# from skimage import io, metrics

# # Load the original and processed images
# img_orig = io.imread(xtest[0])
# img_proc = io.imread(pred[0])
# # 
# # Calculate the MSE and PSNR
# mse = metrics.mean_squared_error(img_orig, img_proc)
# psnr = metrics.peak_signal_noise_ratio(img_orig, img_proc)

# # Calculate the SSIM
# ssim = metrics.structural_similarity(img_orig, img_proc, multichannel=True)

# # Print the results
# print('MSE:',mse)
# print('PSNR:',psnr)
# print('SSIM:',ssim)



In [ ]:
#pip install --upgrade scikit-image
